In [8]:
import torch
from experiments.image.model.dense_flow import DenseFlow
from torchvision import transforms
from PIL import Image
from torchvision.transforms.functional import to_pil_image, to_tensor


In [2]:
# 3.1 Load model
checkpoint = torch.load('/BS/data_mani_compress/work/thesis/thesis/external/DenseFlow/denseflow/imn64/imagenet64/check/checkpoint.pt', map_location='cuda')

/tmp/ipykernel_8492/2133285996.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/BS/data_mani_compress/work/thesis/thesis/external/DenseFlow/dens

In [3]:
print(checkpoint.keys())
model = DenseFlow(data_shape=(3, 64, 64),                  # ImageNet64
    block_config=[6, 4, 1],
    layers_config=[5, 6, 20],
    layer_mid_chnls=[48, 48, 48],
    growth_rate=10)
model.load_state_dict(checkpoint['model'])

dict_keys(['current_epoch', 'train_metrics', 'eval_metrics', 'eval_epochs', 'model', 'optimizer', 'scheduler_iter', 'scheduler_epoch'])


<All keys matched successfully>

In [6]:
def resize_with_box(tensor_batch, size=(64, 64)):
    resized = []
    for img in tensor_batch:  # assumes shape [B, 3, H, W]
        pil_img = to_pil_image(img)  # converts to PIL Image
        pil_resized = pil_img.resize(size, resample=Image.BOX)
        tensor_resized = to_tensor(pil_resized)  # back to tensor
        resized.append(tensor_resized)
    return torch.stack(resized)

In [5]:
# import imagenet loader
from data.utils.dataloaders import get_imagenet_dataloader
imagenet_loader = get_imagenet_dataloader(batch_size=2)


/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python3.11/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 20, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:
first_instances = next(iter(imagenet_loader)) # list of 2 bc first element holds the images and second element holds the labels
print((first_instances[0][1].shape))
first_images = first_instances[0]  # [batch_size, 3, 64, 64]
print(first_images.shape)

# Assuming `first_images` is your input tensor of shape [B, 3, 256, 256]
first_images_box = resize_with_box(first_images)

# 3.3 Compute log-likelihood
with torch.no_grad():
    log_px = model.log_prob(first_images_box)  # log probability [nats/image]
print(f"Log-likelihood: {log_px} nats")

torch.Size([3, 256, 256])
torch.Size([2, 3, 256, 256])


/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Log-likelihood: tensor([-40381.1289, -41165.4375]) nats
